In [6]:
#imports
import numpy as np
import matplotlib.pyplot as plt
from pressure_conversion import pressure_conversion

In [7]:
#define constants

g       = 9.81              # gravitational acceleration [m/s²]

L       = 1000              # length of pipeline [m]
rho     = 1000              # density of water [kg/m³]
D       = 1                 # pipe diameter [m]
Q0      = 2                 # initial flow in whole pipe [m³/s]
h       = 20                # water level in upstream reservoir [m]
n       = 10                # number of pipe segments in discretization
nt      = 1500              # number of time steps after initial conditions
f_D     = 0.05              # Darcy friction factor
c       = 400               # propagation velocity of the pressure wave [m/s]



In [8]:
# preparing the discretization and initial conditions

dx      = L/n                       # length of each pipe segment
dt      = dx/c                      # timestep according to method of characterisitics
nn      = n+1                       # number of nodes
pl_vec  = np.arange(0,nn*dx,dx)     # pl = pipe-length. position of the nodes on the pipeline
t_vec   = np.arange(0,nt*dt,dt) # time vector

v0 = Q0/(D**2/4*np.pi)
p0 = (rho*g*h-v0**2*rho/2)

# storage vectors for old parameters
v_old = np.full(nn,v0)
p_old = p0-(f_D*pl_vec/D*rho/2*v0**2)   # ref Wikipedia: Darcy Weisbach

# storage vectors for new parameters
v_new = np.zeros_like(v_old)
p_new = np.zeros_like(p_old)

# storage vector for time evolution of parameters at node 1 (at reservoir)
p_1 = np.zeros_like(t_vec)
v_1 = np.zeros_like(t_vec)

# storage vector for time evolution of parameters at node N+1 (at valve)
p_np1 = np.zeros_like(t_vec)
v_np1 = np.zeros_like(t_vec)



In [9]:
%matplotlib qt
# plotting preparation

fig1,axs1 = plt.subplots(2,1)
axs1[0].set_title('Pressure distribution in pipeline')
axs1[1].set_title('Velocity distribution in pipeline')

lo_00, = axs1[0].plot(pl_vec,p_old,marker='.')
lo_01, = axs1[1].plot(pl_vec,v_old,marker='.')
axs1[0].set_ylim([-20*p0,20*p0])
axs1[1].set_ylim([-2*v0,2*v0])
fig1.tight_layout()
plt.pause(1)


In [10]:
for it in range(1,nt):

    # set boundary conditions
    v_new[-1]   = 0       # in front of the instantaneously closing valve, the velocity is 0
    p_new[0]    = p0      # hydrostatic pressure from the reservoir

    # calculate the new parameters at first and last node
    v_new[0]    = v_old[1]+1/(rho*c)*(p0-p_old[1])-f_D*dt/(2*D)*abs(v_old[1])*v_old[1]
    p_new[-1]   = p_old[-2]+rho*c*v_old[-2]-rho*c*f_D*dt/(2*D) *abs(v_old[-2])*v_old[-2]

    # calculate parameters at second to second-to-last nodes 
    #equation 2-30 plus 2-31 (and refactor for v_i^j+1) in block 2

    for i in range(1,nn-1):
        v_new[i] = 0.5*(v_old[i-1]+v_old[i+1])+0.5/(rho*c)*(p_old[i-1]-p_old[i+1]) \
            -f_D*dt/(4*D)*(abs(v_old[i-1])*v_old[i-1]+abs(v_old[i+1])*v_old[i+1])

        p_new[i] = 0.5*rho*c*(v_old[i-1]-v_old[i+1])+0.5*(p_old[i-1]+p_old[i+1]) \
            -rho*c*f_D*dt/(4*D)*(abs(v_old[i-1])*v_old[i-1]-abs(v_old[i+1])*v_old[i+1])
    

    # prepare for next loop
    # use .copy() to avoid that memory address is overwritten and hell breaks loose :D
        #https://www.geeksforgeeks.org/array-copying-in-python/
    p_old = p_new.copy()
    v_old = v_new.copy()


    lo_00.set_ydata(p_new)
    lo_01.set_ydata(v_new)
    
    fig1.suptitle(str(it))
    fig1.canvas.draw()
    fig1.tight_layout()
    plt.pause(0.1)

    # store parameters of node 1 (at reservoir)
    p_1[it] = p_new[0]
    v_1[it] = v_new[0]
    # store parameters of node N+1 (at reservoir)
    p_np1[it] = p_new[-1]
    v_np1[it] = v_new[-1]